# GAN with raw audio + Xenakis' screen

02.03.2020 
dataset is uploaded in chunks of size (2048,) --> takes a long time to process
generator : make screen to use as input

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Flatten, Dense, LeakyReLU, ReLU, BatchNormalization, Dropout, Conv1D, AveragePooling1D, Reshape
import glob
import soundfile as sf

In [17]:
#Find a way to make a datasets of numpy arrays 
def load_audio():
    for filepath in glob.iglob(r'C:\Users\jacin\Documents\dataset\*.flac'):
        data = sf.read(filepath)  # convert to array
        transpose = np.transpose(data)
    return data

In [18]:
audio = load_audio()
audio

(array([[ 4.79221344e-05,  3.29017639e-05],
        [ 3.52859497e-05, -3.96966934e-05],
        [-5.24520874e-06, -2.51531601e-05],
        ...,
        [ 7.03692436e-04, -6.03318214e-04],
        [-8.44001770e-05, -9.87052917e-04],
        [ 1.48057938e-04,  1.43766403e-04]]),
 96000)

In [6]:
new_audio =  np.hsplit(audio, 5)

In [9]:
new_audio = np.asarray(new_audio)

In [11]:
new_audio.shape

(5, 2, 3157403)

In [12]:
BATCH_SIZE = 16
BUFFER_SIZE = 16000

In [1]:
#numpy array -> dataset
train_dataset = tf.data.Dataset.from_tensor_slices(audio)
train_dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE)

NameError: name 'tf' is not defined

 generator distribution with pyo http://ajaxsoundstudio.com/pyodoc/index.html see pyo.ipynb 

In [7]:
# Generator Model
def make_generator_model():
    model = Sequential()  
    model.add(Dense(2048, input_shape=(200,))) 
    model.add(LeakyReLU(alpha=0.01)) 
    model.add(BatchNormalization(momentum=0.9)) 
    model.add(Reshape((16, 1024)))

    model.add(Conv1D(16, 1024, padding='same'))
    model.add(ReLU())
    model.add(BatchNormalization(momentum=0.9))
    model.add(Dropout(rate=0.1))

    model.add(Conv1D(64, 512, padding='same'))
    model.add(ReLU())
    model.add(BatchNormalization(momentum=0.9))
    model.add(Dropout(rate=0.1))

    model.add(Conv1D(256, 256, padding='same'))
    model.add(ReLU())
    model.add(BatchNormalization(momentum=0.9))
    model.add(Dropout(rate=0.1))
    
    model.add(Conv1D(1024, 128, padding='same'))
    model.add(Dropout(rate=0.3))
    model.summary()
    return model

In [ ]:
generator = make_generator_model()

noise = tf.random.normal([1, 2048])
generated_sound = generator(noise, training=False)

In [1]:
# Discriminator Model
def make_discriminator_model():
    model = Sequential()
    model.add(Conv1D(256, 256, padding='valid')) 
    model.add(LeakyReLU(alpha=0.01)) 
    model.add(BatchNormalization(momentum=0.9))
    
    model.add(Conv1D(64, 512, strides=4, padding='valid'))
    model.add(ReLU())
    model.add(AveragePooling1D(4))
    model.add(BatchNormalization(momentum=0.9))
    
    model.add(Conv1D(16, 1024, strides=4, padding='valid'))
    model.add(ReLU())
    model.add(AveragePooling1D(4))
    model.add(BatchNormalization(momentum=0.9))
    model.add(Dropout(rate=0.1))
  
    model.add(Flatten())
    model.add(LeakyReLU(alpha=0.01))
    model.add(BatchNormalization(momentum=0.9))
    model.add(Dense(1, activation='sigmoid'))
    model.summary()
    return model

In [2]:
discriminator = make_discriminator_model()
decision = discriminator(generated_sound)
print (decision)

NameError: name 'tf' is not defined

In [ ]:
# This method returns a helper function to compute cross entropy loss
cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits=True)

In [18]:
def discriminator_loss(real_output, fake_output):
    real_loss = cross_entropy(tf.ones_like(real_output), real_output)
    fake_loss = cross_entropy(tf.zeros_like(fake_output), fake_output)
    total_loss = real_loss + fake_loss
    return total_loss

In [19]:
def generator_loss(fake_output):
    return cross_entropy(tf.ones_like(fake_output), fake_output)

In [20]:
generator_optimizer = tf.keras.optimizers.Adam(1e-4)
discriminator_optimizer = tf.keras.optimizers.Adam(1e-4)

In [21]:
import os
checkpoint_dir = './training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")
checkpoint = tf.train.Checkpoint(generator_optimizer=generator_optimizer,
                                 discriminator_optimizer=discriminator_optimizer,
                                 generator=generator,
                                 discriminator=discriminator)

In [22]:
EPOCHS = 50
noise_dim = 500
num_examples_to_generate = 16

seed = tf.random.normal([num_examples_to_generate, noise_dim])

In [23]:
# Notice the use of `tf.function`
# This annotation causes the function to be "compiled".
@tf.function
def train_step(training_data):
    noise = tf.random.normal([BATCH_SIZE, noise_dim])

    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
        generated_sounds = generator(noise, training=True)

        real_output = discriminator(training_data, training=True)
        fake_output = discriminator(generated_sounds, training=True)

        gen_loss = generator_loss(fake_output)
        disc_loss = discriminator_loss(real_output, fake_output)

    gradients_of_generator = gen_tape.gradient(gen_loss, generator.trainable_variables)
    gradients_of_discriminator = disc_tape.gradient(disc_loss, discriminator.trainable_variables)

    generator_optimizer.apply_gradients(zip(gradients_of_generator, generator.trainable_variables))
    discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, discriminator.trainable_variables))

In [24]:
def train(dataset, epochs):
    for epoch in range(epochs):
        start = time.time()

        for batch in dataset:
            train_step(batch)

    # Save the model every 15 epochs
    if (epoch + 1) % 15 == 0:
          checkpoint.save(file_prefix = checkpoint_prefix)

    print ('Time for epoch {} is {} sec'.format(epoch + 1, time.time()-start))

      # Generate after the final epoch
    display.clear_output(wait=True)
    generate_and_save_sounds(generator, epochs, seed)

In [25]:
def generate_and_save_sounds(model, epoch, test_input):
  # Notice `training` is set to False.
  # This is so all layers run in inference mode (batchnorm).
    predictions = model(test_input, training=False)

    for i in range(predictions.shape[0]):
        new_audio = sf.write('new_file.flac', predictions[i], samplerate)
    return new_audio

In [26]:
import time

In [27]:
train(train_dataset, EPOCHS)


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.



ValueError: in converted code:

    <ipython-input-23-f4bc3e375100>:10 train_step  *
        real_output = discriminator(training_data, training=True)
    C:\Users\jacin\Miniconda3\envs\tf_gpu\lib\site-packages\tensorflow_core\python\keras\engine\base_layer.py:847 __call__
        outputs = call_fn(cast_inputs, *args, **kwargs)
    C:\Users\jacin\Miniconda3\envs\tf_gpu\lib\site-packages\tensorflow_core\python\keras\engine\sequential.py:256 call
        return super(Sequential, self).call(inputs, training=training, mask=mask)
    C:\Users\jacin\Miniconda3\envs\tf_gpu\lib\site-packages\tensorflow_core\python\keras\engine\network.py:708 call
        convert_kwargs_to_constants=base_layer_utils.call_context().saving)
    C:\Users\jacin\Miniconda3\envs\tf_gpu\lib\site-packages\tensorflow_core\python\keras\engine\network.py:860 _run_internal_graph
        output_tensors = layer(computed_tensors, **kwargs)
    C:\Users\jacin\Miniconda3\envs\tf_gpu\lib\site-packages\tensorflow_core\python\kera